<a href="https://colab.research.google.com/github/Vasugi2003/Students-Attendance-Monitoring-System/blob/main/STUDENTS_ATTENDANCE_TRACKING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2

In [ ]:
%%cmd
where python

Microsoft Windows [Version 10.0.19045.3570]
(c) Microsoft Corporation. All rights reserved.

C:\Users\RITHANYA>where python
C:\Users\RITHANYA\anaconda3\anaconda\python.exe
C:\Python311\python.exe
C:\Users\RITHANYA\anaconda3\python.exe
C:\Users\RITHANYA\AppData\Local\Microsoft\WindowsApps\python.exe

C:\Users\RITHANYA>

In [ ]:
%%cmd
pip install face-recognition


Microsoft Windows [Version 10.0.19045.3570]
(c) Microsoft Corporation. All rights reserved.

C:\Users\RITHANYA>pip install face-recognition

C:\Users\RITHANYA>


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.19045.3570]
(c) Microsoft Corporation. All rights reserved.

C:\Users\RITHANYA>pip install cmake

C:\Users\RITHANYA>


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%%cmd
python --version

Microsoft Windows [Version 10.0.19045.3570]
(c) Microsoft Corporation. All rights reserved.

C:\Users\RITHANYA>python --version
Python 3.9.13

C:\Users\RITHANYA>

In [ ]:
%%cmd
pip install "C:\Users\RITHANYA\OneDrive\Desktop\dlib-19.22.99-cp39-cp39-win_amd64.whl"

Microsoft Windows [Version 10.0.19045.3570]
(c) Microsoft Corporation. All rights reserved.

C:\Users\RITHANYA>pip install "C:\Users\RITHANYA\OneDrive\Desktop\dlib-19.22.99-cp39-cp39-win_amd64.whl"
Processing c:\users\rithanya\onedrive\desktop\dlib-19.22.99-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

C:\Users\RITHANYA>


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%%cmd
pip install face-recognition

Microsoft Windows [Version 10.0.19045.3570]
(c) Microsoft Corporation. All rights reserved.

C:\Users\RITHANYA>pip install face-recognition

C:\Users\RITHANYA>


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import face_recognition
import os
import glob
import numpy as np
import csv
import datetime  # Import the datetime module

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names

    # ... (rest of the class remains the same)

# Create an instance of SimpleFacerec
sfr = SimpleFacerec()
sfr.load_encoding_images("C:/Users/RITHANYA/OneDrive/Desktop/SPARK/images")

cap1 = cv2.VideoCapture("C:/Users/RITHANYA/OneDrive/Desktop/SPARK/camera2.mp4")  # Update with actual path
cap2 = cv2.VideoCapture("C:/Users/RITHANYA/OneDrive/Desktop/SPARK/final.mp4")  # Update with actual path

if not cap1.isOpened() or not cap2.isOpened():
    print("Error opening video files.")
    exit()

# Initialize the CSV file and write headers
csv_filename = "detected_faces.csv"
with open(csv_filename, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Timestamp", "Name", "Camera"])

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        if cap1.isOpened() and cap2.isOpened():
            print("Error reading frames.")
        else:
            print("End of videos.")
        break

    face_locations1, face_names1 = sfr.detect_known_faces(frame1)
    face_locations2, face_names2 = sfr.detect_known_faces(frame2)

    # Get the current timestamp
    current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    with open(csv_filename, "a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        for name in face_names1:
            csv_writer.writerow([current_timestamp, name, "camera1"])
        for name in face_names2:
            csv_writer.writerow([current_timestamp, name, "camera2"])

    for face_loc, name in zip(face_locations1, face_names1):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
        cv2.putText(frame1, name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame1, (x1, y1), (x2, y2), (0, 0, 200), 2)

    for face_loc, name in zip(face_locations2, face_names2):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
        cv2.putText(frame2, name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame2, (x1, y1), (x2, y2), (0, 0, 200), 2)

    cv2.imshow("Camera 1", frame1)
    cv2.imshow("Camera 2", frame2)

    key = cv2.waitKey(1)
    if key == 27:
        break
cap1.release()
cap2.release()

cv2.destroyAllWindows()


13 encoding images found.
Encoding images loaded


In [ ]:
import cv2
import face_recognition
import os
import glob
import numpy as np
import csv
import datetime

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.75

    def load_encoding_images(self, images_path):
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names

# Create an instance of SimpleFacerec
sfr = SimpleFacerec()
sfr.load_encoding_images("C:/Users/RITHANYA/OneDrive/Desktop/SPARK/images")

cap1 = cv2.VideoCapture("C:/Users/RITHANYA/OneDrive/Desktop/SPARK/final.mp4")  # Update with actual path
cap2 = cv2.VideoCapture("C:/Users/RITHANYA/OneDrive/Desktop/SPARK/camera2.mp4")  # Update with actual path

if not cap1.isOpened() or not cap2.isOpened():
    print("Error opening video files.")
    exit()

# Initialize the CSV file and write headers
csv_filename = "detected_faces.csv"
with open(csv_filename, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Timestamp", "Name", "Camera"])

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        if cap1.isOpened() and cap2.isOpened():
            print("Error reading frames.")
        else:
            print("End of videos.")
        break

    face_locations1, face_names1 = sfr.detect_known_faces(frame1)
    face_locations2, face_names2 = sfr.detect_known_faces(frame2)

    # Get the current timestamp
    current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    with open(csv_filename, "a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        for name in face_names1:
            csv_writer.writerow([current_timestamp, name, "camera1"])
        for name in face_names2:
            csv_writer.writerow([current_timestamp, name, "camera2"])

    for face_loc, name in zip(face_locations1, face_names1):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

        # Display name and timestamp
        text = f"{name} ({current_timestamp})"

        cv2.putText(frame1, text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame1, (x1, y1), (x2, y2), (0, 0, 200), 2)

    for face_loc, name in zip(face_locations2, face_names2):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

        # Display name and timestamp
        text = f"{name} ({current_timestamp})"

        cv2.putText(frame2, text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame2, (x1, y1), (x2, y2), (0, 0, 200), 2)

    cv2.imshow("Camera 1", frame1)
    cv2.imshow("Camera 2", frame2)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap1.release()
cap2.release()

cv2.destroyAllWindows()


13 encoding images found.
Encoding images loaded
